In [1]:
import pandas as pd
import import_ipynb
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori


#Methoden
import Preprocessing
import FrequentItemsets

importing Jupyter notebook from Preprocessing.ipynb
importing Jupyter notebook from FrequentItemsets.ipynb


In [2]:
class FahrzeugModell:




    gg_id = 261
    mj_4 = 2020
    btyp = ''

    def __init__(self, btyp):
        self.btyp = btyp
        
        self.orders = pd.read_csv("Daten/Februar2021/orders.csv", delimiter=";")
        self.orders_options = pd.read_csv("Daten/Februar2021/orders_options.csv", delimiter=";")
        self.option_prices = pd.read_csv("Daten/option_package_prices.csv", delimiter=";")
        self.btyp_prices = pd.read_csv("Daten/model_base_price.csv" , delimiter=";")
        self.abzug = pd.read_csv("Daten/s_abzug_po.csv" , delimiter=";")
        self.metadata_china = pd.read_csv("Daten/2021-02-28_China__PCN__C33_ExportExcel-2020.csv",
                                          delimiter=";", encoding="latin1")

    def get_Big(self):
        
        
        
        orders_filtered = Preprocessing.filtering(self.orders, self.gg_id, self.btyp, self.mj_4)
        data_combined = Preprocessing.preprocess_datacombined(orders_filtered, self.orders_options)
        option_prices_filtered = Preprocessing.filtering(self.option_prices, self.gg_id, self.btyp, self.mj_4)
        btyp_prices_filtered = Preprocessing.filtering(self.btyp_prices, self.gg_id, self.btyp, self.mj_4)
        frequent_itemsets = FrequentItemsets.get_frequentitemsets(data_combined)
       
        support_option_price = pd.merge(option_prices_filtered, frequent_itemsets, left_on="option_key", right_on="itemsets")
        support_price = pd.merge(btyp_prices_filtered, support_option_price, on="btyp")
        abzug1 = self.abzug[self.abzug['btyp']==self.btyp]
        data = pd.merge(support_price, abzug1, left_on='ausstatt_alt', right_on='aus', how='inner', suffixes=('', '_y'))
        data.drop(data.filter(regex='_y$').columns.tolist(),axis=1, inplace=True)  
        metadata_china1 = self.metadata_china[(self.metadata_china['Language'] =='CE') & (self.metadata_china['Model Code'] == self.btyp)]
        big_data = pd.merge(data, metadata_china1, left_on='ausstatt_alt', right_on= 'Equipment', how= 'inner')
        
        return big_data[['preis_typ_x', 'gr_prs', 'waehr_kz_x', 'option_key', 'preis', 'support',
                        'm_familie', 'Model Year', 'Model Code', 'Category', 'Equipment',
                        'Text', 'Longtext']]
    
